<a href="https://colab.research.google.com/github/yukinaga/bayesian_statistics/blob/main/section_4/03_ab_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyMC3によるA/Bテスト
PyMC3を使い、A/Bテストの結果を分析します。  
ボタンを2種類用意し、ウェブサイト上でランダムに入れ替えます。  
どちらのクリック率が高いのか、ベイズ推定を使って分析しましょう。  

## pymc3のインストール
pymc3をpipによりインストールします。

In [ ]:
!pip install pymc3

必要な各設定を行います。

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pymc3 as pm

plt.style.use("seaborn-darkgrid")

## 統計モデルの構築とサンプリング
PyMC3により、「統計モデル」を構築し、事後分布に従ったサンプリングを行います。

In [ ]:
n = 50  # ボタンの表示数
clicks = 4  # クリック数

with pm.Model() as model:
    theta = pm.Uniform("theta", lower=0, upper=1)  # 事前分布: 一様分布
    likehood = pm.Binomial("likehood", p=theta, n=n, observed=clicks)  # 尤度: 二項分布
    trace = pm.sample(4000, chains=2, return_inferencedata=False)  # サンプリング

`pm.plot_trace()`により、事後分布からサンプリングした結果を可視化することができます。

In [ ]:
with model:
    pm.plot_trace(trace)

`pm.summary()`によるモデルの概要を表示することができます。  
`hdi_prob=0.95`と記述すると、確率が上位95%の区間が表示されます。

In [ ]:
with model:
    print(pm.summary(trace, hdi_prob=0.95))

## 事後分布の表示
`plot_posterior`により、PyMC3によって推論された事後分布がグラフで表示されます。

In [ ]:
with model:
    pm.plot_posterior(trace, hdi_prob=0.95)

## A/Bテスト
PyMC3では、複数のデータのグループで同時に統計モデルの構築、サンプリングを行うことができます。  
これを利用し、2種類のボタンでA/Bテストを行います。

In [ ]:
n = (30, 50)  # ボタンの表示数
clicks = (2, 5)  # クリック数

with pm.Model() as model:
    theta = pm.Uniform("theta", lower=0, upper=1, shape=2)
    likehood = pm.Binomial("likehood", p=theta, n=n, observed=clicks)
    trace = pm.sample(4000, chains=2, return_inferencedata=False)

`pm.plot_trace()`によりサンプリング結果を可視化します。

In [ ]:
with model:
    pm.plot_trace(trace)

また、片方のボタンがもう片方のボタンに比べてクリック率が高い割合を計算します。

In [ ]:
with model:
    print((trace["theta"][:, 1] - trace["theta"][:, 0] > 0).mean())

この場合、2つのボタンに大きなクリック率の違いはありません。  
次に、試行数を増やします。  

In [ ]:
n = (1500, 2100)  # ボタンの表示数
clicks = (98, 189)  # クリック数

with pm.Model() as model:
    theta = pm.Uniform("theta", lower=0, upper=1, shape=2)
    obs = pm.Binomial("likehood", p=theta, n=n, observed=clicks)
    trace = pm.sample(4000, chains=2, return_inferencedata=False)

`pm.plot_trace()`によりサンプリング結果を可視化します。

In [ ]:
with model:
    pm.plot_trace(trace)

片方のボタンがもう片方のボタンに比べてクリック率が高い割合を計算します。

In [ ]:
print((trace["theta"][:, 1] - trace["theta"][:, 0] > 0).mean())

2つのボタンに明らかなクリック率の違いが確認できます。  
試行数を増やすことで分布はシャープになり、その結果クリック率の違いが明瞭になりました。